In [8]:
import re
import shutil
import github3
from github3 import login
from git import Repo, GitCommandError
import yaml
from subprocess import Popen, PIPE, STDOUT, SubprocessError, check_output
from configparser import ConfigParser
%cd ~/Documents/projects/nerve/python/
from nerve.core.model import Nerve
from nerve.core.client import Client
from nerve.core.git import Git
from nerve.core.git_lfs import GitLFS
from nerve.core.utils import execute_subprocess
from nerve.test.test_project import *
%cd ~/Documents/projects/nerve

/Users/alex/Documents/projects/nerve/python
/Users/alex/Documents/projects/nerve


In [16]:
config_path = '/Users/alex/Documents/projects/nerve/python/nerve/resources/nerverc.yml'
nerve = Nerve(config_path)
name = 'nerve-test-repo-2'
proj = os.path.join(nerve['project-root'], name)
working_dir = '/Volumes/data/nerve-test-projects/nerve-test-repo-2'
config = nerve.config
del config['gitignore']
pprint(config)

{'assets': ['maya'],
 'deliverables': ['vol001', 'geo001'],
 'extensions': ['obj', 'png'],
 'organization': 'nerve-test-organization',
 'private': True,
 'project-root': '/private/tmp',
 'publish-exclude-patterns': None,
 'publish-include-patterns': ['geo.*'],
 'request-exclude-patterns': None,
 'request-include-patterns': ['vol.*'],
 'specification': 'proj001',
 'teams': {'biz': 'read', 'dev': 'write'},
 'token': 'e17afcc8c42fe24536c14700c4931f0e7f252bb1',
 'url-type': 'ssh',
 'user-branch': 'alex',
 'username': 'theNewFlesh'}


In [17]:
test_whole_project_cycle()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Applications/anaconda/envs/nerve-dev/lib/python3.4/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-17-efee4d34e0b2>", line 1, in <module>
    test_whole_project_cycle()
  File "/Users/alex/Documents/projects/nerve/python/nerve/test/test_project.py", line 7, in test_whole_project_cycle
    config = os.path.join(os.getcwd(), 'python', 'nerve', 'resources', 'nerverc.yml')
FileNotFoundError: [Errno 2] No such file or directory

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Applications/anaconda/envs/nerve-dev/lib/python3.4/site-packages/IPython/core/interactiveshell.py", line 1821, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'FileNotFoundError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

FileNotFoundError: [Errno 2] No such file or directory

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
lrwxr-xr-x@ 1 root  wheel  11 Mar 21  2016 /tmp@ -> private/tmp


In [13]:
def patch(name, nerve, user, username):
    path = '/Volumes/data/nerve-test-projects/' + user
    if not os.path.exists(path):
        os.mkdir(path)
    nerve._config['project-root'] = '/Volumes/data/nerve-test-projects/' + user
    nerve._config['user-branch'] = user
    nerve._config['username'] = username
    nerve.clone(name)

branches = [
#     ('alex', 'theNewFlesh'),
    ('nerve-test-user', 'nerve-test-user')
]
for user, username in branches:
    patch(name, nerve, user, username)

In [9]:
def create_command(branch, version, asset):
    cmd = 'echo v{v} > {a}001/ntr_{a}_v{v}.txt; git add --all; git commit -m "v{v}";'
#     cmd += ' git push origin {b}'
    cmd = cmd.format(a=asset, b=branch, v=version)
    return cmd

def do(branch, versions, asset):
    cmds = []    
    cmds.append('git checkout {b}'.format(b=branch))
    cmds.append('git pull origin dev:{b}'.format(b=branch))
    for ver in versions:
        cmds.append(create_command(branch, ver, asset))
    cmds.append('git push origin {b}'.format(b=branch))
    path = '/Users/alex/Documents/data/nerve-test-projects/' + branch + '/nerve-test-repo'
#     os.chdir(path)
    for cmd in cmds: 
        print(cmd)
#         execute_subprocess(cmd)

In [10]:
do('nerve-test-user', ['001', '002'], 'vol')
# do('nerve-test-user', ['001', '002'], 'geo')
# do('alex', ['003', '004'], 'vol')
# do('nerve-test-user', ['003', '004'], 'geo')

git checkout nerve-test-user
git pull origin dev:nerve-test-user
echo v001 > vol001/ntr_vol_v001.txt; git add --all; git commit -m "v001";
echo v002 > vol001/ntr_vol_v002.txt; git add --all; git commit -m "v002";
git push origin nerve-test-user


In [7]:
!git checkout dev; git merge nerve-test-user -m 'blah'; git push origin dev; git checkout nerve-test-user

Switched to branch 'dev'
Your branch is up-to-date with 'origin/dev'.
Updating 6900604..390a4da
Fast-forward (no commit created; -m option ignored)
 vol001/ntr_vol_v001.txt | 1 +
 vol001/ntr_vol_v002.txt | 1 +
 2 files changed, 2 insertions(+)
 create mode 100644 vol001/ntr_vol_v001.txt
 create mode 100644 vol001/ntr_vol_v002.txt
Total 0 (delta 0), reused 0 (delta 0)
To git@github.com:nerve-test-organization/nerve-test-repo.git
   6900604..390a4da  dev -> dev


In [15]:
do('alex', ['001', '002'], 'geo')

git checkout alex
git pull origin dev:alex
echo v001 > geo001/ntr_geo_v001.txt; git add --all; git commit -m "v001";
echo v002 > geo001/ntr_geo_v002.txt; git add --all; git commit -m "v002";
git push origin alex


In [10]:
!git checkout dev; git merge alex -m 'blah'; git push origin dev; git checkout alex

Switched to branch 'dev'
Your branch is behind 'origin/dev' by 2 commits, and can be fast-forwarded.
  (use "git pull" to update your local branch)
Updating 6900604..39e8196
Fast-forward (no commit created; -m option ignored)
 geo001/ntr_geo_v001.txt | 1 +
 geo001/ntr_geo_v002.txt | 1 +
 vol001/ntr_vol_v001.txt | 1 +
 vol001/ntr_vol_v002.txt | 1 +
 4 files changed, 4 insertions(+)
 create mode 100644 geo001/ntr_geo_v001.txt
 create mode 100644 geo001/ntr_geo_v002.txt
 create mode 100644 vol001/ntr_vol_v001.txt
 create mode 100644 vol001/ntr_vol_v002.txt
Total 0 (delta 0), reused 0 (delta 0)
To git@github.com:nerve-test-organization/nerve-test-repo.git
   390a4da..39e8196  dev -> dev
Switched to branch 'alex'


In [8]:
do('alex', ['003', '004'], 'vol')
do('dev', ['003', '004'], 'geo')

git checkout alex
echo v003 > vol001/ntr_vol_v003.txt; git add --all; git commit -m "v003";
echo v004 > vol001/ntr_vol_v004.txt; git add --all; git commit -m "v004";
git push origin alex
git checkout dev
echo v003 > geo001/ntr_geo_v003.txt; git add --all; git commit -m "v003";
echo v004 > geo001/ntr_geo_v004.txt; git add --all; git commit -m "v004";
git push origin dev


In [17]:
username = 'theNewFlesh'
name = 'nerve-test-repo'
token = 'e17afcc8c42fe24536c14700c4931f0e7f252bb1'
client = login(username, token=token)
orgname = 'nerve-test-organization'
org = client.organization(orgname)
repo = client.repository(orgname, name)
# {team.id: team.name for team in org.teams()}

In [22]:
uri = 'git@github.com:nerve-test-organization/nerve-test-repo.git'
path = '/Users/alex/Documents/data/nerve-test-projects/temp/nerve-test-repo'
x = Repo.clone_from(uri, path, branch='dev')